In [1]:
from utils import *
from coco_dataset import *
from tqdm import tqdm
from pprint import PrettyPrinter

In [2]:
pp = PrettyPrinter()

# Parameters
path2data = "./data/val2017"
path2json = "./data/annotations/instances_val2017.json"
batch_size = 64
workers = 4
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
checkpoint = './Checkpoint_temp/checkpoint_epoch_67_ssd300.pth.tar'

# Load model to be evaluated
checkpoint = torch.load(checkpoint)
model = checkpoint['model']

model = model.to(device)

# eval mode
model.eval()

# Load test data
test_dataset = CocoDetectionForSSD(root=path2data, annFile=path2json, transform=None, target_transform=None, split='TEST')
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = batch_size, shuffle = False, collate_fn = collate_fn_SSD, num_workers = workers, pin_memory = True)

loading annotations into memory...
Done (t=0.33s)
creating index...
index created!


In [3]:
det_boxes = []
det_labels = []
det_scores = []
true_boxes = []
true_labels = []
def evaluate(test_loader, model):
    model.eval()
    

    
    with torch.no_grad():
        for i, (images, boxes, labels) in enumerate(tqdm(test_loader, desc = 'Evaluating')):
            images = images.to(device)
            boxes = [b.to(device) for b in boxes]
            labels = [l.to(device) for l in labels]
            
            predicted_locs, predicted_scores = model(images)
            # Use paper's paramters
            det_boxes_batch, det_labels_batch, det_scores_batch = model.detect_objects(predicted_locs, predicted_scores, min_score = 0.1, max_overlap = 0.45, top_k = 200)
            det_boxes.extend(det_boxes_batch)
            det_labels.extend(det_labels_batch)
            det_scores.extend(det_scores_batch)
            true_boxes.extend(boxes)
            true_labels.extend(labels)
            #print('true_boxes:', boxes)
            #print('det_boxes:', det_boxes_batch)
        # Calculate mAP
        APs, mAP = calculate_mAP(det_boxes, det_labels, det_scores, true_boxes, true_labels)
        
    pp.pprint(APs)
    print('\nMean Average Precision (mAP): %.3f' % mAP)

In [4]:
evaluate(test_loader, model)

Evaluating: 100%|██████████| 79/79 [01:42<00:00,  1.29s/it]


{'airplane': 0.07508929073810577,
 'apple': 0.0024302396923303604,
 'backpack': 0.0,
 'banana': 0.01503781694918871,
 'baseball bat': 0.0,
 'baseball glove': 0.0,
 'bear': 0.07379777729511261,
 'bed': 0.2172403335571289,
 'bench': 0.016800934448838234,
 'bicycle': 0.0050927200354635715,
 'bird': 0.005614848807454109,
 'boat': 0.00977528840303421,
 'book': 0.00459744967520237,
 'bottle': 0.00011223344336031005,
 'bowl': 0.0033805607818067074,
 'broccoli': 0.0016573603497818112,
 'bus': 0.04476691782474518,
 'cake': 0.0050407471135258675,
 'car': 0.0003120143956039101,
 'carrot': 0.001946154865436256,
 'cat': 0.07280906289815903,
 'cell phone': 0.003223513485863805,
 'chair': 0.0005442775436677039,
 'clock': 0.00959749985486269,
 'couch': 0.011459285393357277,
 'cow': 0.011504564434289932,
 'cup': 1.6090103599708527e-05,
 'dining table': 0.1527860462665558,
 'dog': 0.036883674561977386,
 'donut': 0.0002469260070938617,
 'elephant': 0.03966738283634186,
 'fire hydrant': 0.0327346175909042

In [2]:
det_boxes = [torch.tensor([[ 50.8456,  11.0575, 497.9483, 319.0857],[ 50.8456,  11.0575, 497.9483, 319.0857]])]
det_labels = [torch.tensor([1,1])]
det_scores = [torch.tensor([1,1])]
true_boxes = [torch.tensor([[ 67.9887, 155.5200, 276.2039, 240.4080],
                                      [ 11.3314,   7.7760, 498.5836, 322.7040]])]
true_labels = [torch.tensor([5, 1])]

calculate_mAP(det_boxes, det_labels, det_scores, true_boxes, true_labels)

cumul_TP: tensor([1., 1.], device='cuda:0')
cumul_FP: tensor([0., 1.], device='cuda:0')
cumul_precision: tensor([1.0000, 0.5000], device='cuda:0')
cumul_recall: tensor([1., 1.], device='cuda:0')


({'person': 1.0,
  'bicycle': 0.0,
  'car': 0.0,
  'motorcycle': 0.0,
  'airplane': 0.0,
  'bus': 0.0,
  'train': 0.0,
  'truck': 0.0,
  'boat': 0.0,
  'traffic light': 0.0,
  'fire hydrant': 0.0,
  'stop sign': 0.0,
  'parking meter': 0.0,
  'bench': 0.0,
  'bird': 0.0,
  'cat': 0.0,
  'dog': 0.0,
  'horse': 0.0,
  'sheep': 0.0,
  'cow': 0.0,
  'elephant': 0.0,
  'bear': 0.0,
  'zebra': 0.0,
  'giraffe': 0.0,
  'backpack': 0.0,
  'umbrella': 0.0,
  'handbag': 0.0,
  'tie': 0.0,
  'suitcase': 0.0,
  'frisbee': 0.0,
  'skis': 0.0,
  'snowboard': 0.0,
  'sports ball': 0.0,
  'kite': 0.0,
  'baseball bat': 0.0,
  'baseball glove': 0.0,
  'skateboard': 0.0,
  'surfboard': 0.0,
  'tennis racket': 0.0,
  'bottle': 0.0,
  'wine glass': 0.0,
  'cup': 0.0,
  'fork': 0.0,
  'knife': 0.0,
  'spoon': 0.0,
  'bowl': 0.0,
  'banana': 0.0,
  'apple': 0.0,
  'sandwich': 0.0,
  'orange': 0.0,
  'broccoli': 0.0,
  'carrot': 0.0,
  'hot dog': 0.0,
  'pizza': 0.0,
  'donut': 0.0,
  'cake': 0.0,
  'chair':